In [1]:
from astroquery.ned import Ned 
from astroquery.sdss import SDSS 
#import astroquery as aq
from astropy import coordinates as coords   #Conversion, sistemas y marcos de referencia
from astropy import units as u   #Conversion y desarrollo de operaciones aritméticas de instancias Quantity
from astropy.io import fits   
from astropy.table import Table
import numpy as np
#from matplotlib import pyplot as plt
import math as mt
#import sys

/usr/lib/python2.7/site-packages/astroquery/sdss/__init__.py:33: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


###   NED z query and SDSS spectra query

In [4]:
filename = 'newObj'
newList = Table.read(filename+'.txt', format = 'ascii')
newList

col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
int64,int64,int64,float64,float64,string8,float64,int64,int64,string40,int64
1,--,299493525735630848,179.89005384,54.21599414,S,0.0006005246,--,--,LINER,1


In [11]:
newResultListDT=Table(names=('ObjID','ra','dec','z', 'isFits'),dtype=('S33', 'f12', 'f12','f8', 'i1'))
ListaGalaxias=newList['col3']
for i in range(len(ListaGalaxias)):
    name = newList['col3'][i]
    # resultNED= Ned.query_object(ListaGalaxias[i])
    RA = newList['col4'][i]  #*u.degree   #resultNED['RA(deg)'][0]*u.degree   
    DEC = newList['col5'][i] #*u.degree   #resultNED['DEC(deg)'][0]*u.degree
    z = newList['col7'][i]    #resultNED['Redshift'][0]
    pos=coords.SkyCoord(ra=RA, dec=DEC, frame='icrs', unit='deg')  #Cordenadas en sistema ICRS
    try:
        xid = SDSS.query_region(pos, spectro=True)
        if xid is None:
            print i, str(name)+': Not found'
        else:
            sp = SDSS.get_spectra(matches=xid)
            for iii in range(len(sp)):
                hdulist = fits.HDUList(sp[iii])
                total= hdulist[1].data
                linlam=np.exp(total['loglam']*np.log(10))
                lambd = linlam/(1+z)
                if lambd[0]<3700 and lambd[-1]>6800:
                    name = str(ListaGalaxias[i])+'SP'+str(iii+1)
                    print i, str(name)+': Found one spectrum'
                    hdulist.writeto('Exam_fits/'+str(ListaGalaxias[i])+'SP'+str(iii+1)+'.fits',clobber=True)
                    newResultListDT.add_row([name,pos.ra*u.degree,pos.dec*u.degree,z, 1])
    except: # HTTPError:
        # e = sys.exc_info()[0]
        # write_to_page( "<p>Error: %s</p>" % e )
        print "error downloading the file"
        continue

newResultListDT.write(filename+'SDSSFound.tbl', format='ascii')

0 299493525735630848SP1: Found one spectrum


In [6]:
#ListaGalaxias[i]
newResultListDT

ObjID,ra,dec,z,isFits
string264,float32,float32,float64,int8
SDSS J165712.75+321141.4SP1,254.303,32.195,0.038,1


###    NED Spectra query

In [21]:
fileName = 'list1'
#newList = Table.read(fileName+'.txt', format = 'ascii')

['3C234','Arp299Aaa','Arp299Aba','CBS74','IC0860','IC4397','LEDA84081','LEDA84113','LEDA84586','LEDA90159','LEDA90258','Mrk34','Mrk273','Mrk463','Mrk477','Mrk673','Mrk686','Mrk1014','NGC3982aa','NGC3982ba','NGC3982ca','NGC4579aa','NGC4579ba','NGC5033','NGC5273','PGC025218','PGC048898','PGC055020','PGC055114']

newResultListDT=Table(names=('ObjID','ra','dec','z', 'isFits'),dtype=('S33', 'f12', 'f12','f8', 'i1'))
ListaGalaxias = ['3C234','Arp299A','CBS74','IC0860','IC4397','LEDA84081',
                 'LEDA84113','LEDA84586','LEDA90159','LEDA90258','Mrk34','Mrk273','Mrk463',
                 'Mrk477','Mrk673','Mrk686','Mrk1014','NGC3982',
                 'NGC4579','NGC4579','NGC5033','NGC5273','PGC025218','PGC048898','PGC055020','PGC055114']


#['3C234','CBS74','Mrk34','Mrk273','Mrk463','Mrk477','Mrk686','NGC3982','NGC5273','PGC055020','PGC055114']
#['ETACAR', 'M17', 'M16', 'M20','NGC3576' ,'ORION 1 ','ORION 1' ,'S209', 'S283','S98','S127','S128','IC10','N80','N83','N13','N32','N81','N66','NGC295','IC142','NGC595','MA 2','NGC604','NGC588 ','IC131','NGC753','Mkn 600','NGC2363','IIZw40','NGC2366','N59','N44','N55','N113','N127','N159','N4','N214','N79','N191']
#  newList['col3']
for i in range(len(ListaGalaxias)):
    name = ListaGalaxias[i]
    try:
        resultNED= Ned.query_object(name)
        spectra = Ned.get_spectra(name)
    except :
        print "The name",str(name),"cannot be found in NED database."
        continue
    RA = resultNED['RA(deg)'][0]*u.degree
    DEC = resultNED['DEC(deg)'][0]*u.degree
    z = resultNED['Redshift'][0]
    #  newResultListDT.add_row([name,RA,DEC,z, 1])
    if spectra is None:
        print i, str(name)+': Not found'
    else:
        for iii in range(len(spectra)):
            hduli = fits.HDUList(spectra[iii])
            hea= hduli[0].header
            typeFITS=1
            if hea['NAXIS'] == 0:
                typeFITS=0
                print "Can't identify FITS format in file: ", str(name)
                continue
                total= hduli[1].data
            elif hea['NAXIS'] == 1:
                typeFITS=1
            if typeFITS==0:
                spectraFlux = total['flux']
                linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
                lambd = linlam/(1+z)
            else:
                spectraFlux = hduli[0].data
                if 'CDELT1' in hea:
                    lambd = np.linspace(hea['CRVAL1'], hea['CRVAL1']+hea['CDELT1']*(len(spectraFlux)-1), len(spectraFlux))
                elif 'W0' in hea:
                    lambd = np.linspace(hea['W0'], hea['W0']+hea['WPC']*(len(spectraFlux)-1), len(spectraFlux))
                else:
                    print "Can't identify FITS format in file: ", str(name)
                    continue
            lambd = lambd/(1+z)
            if lambd[0]<3700 and lambd[-1]>6800:
                name = str(name)+'SP'+str(iii+1)
                print i, str(name)+': Found one spectrum'
                hduli.writeto('Exam_fits/'+str(name)+'.fits',clobber=True)
                newResultListDT.add_row([name,RA,DEC,z, 1])
                
newResultListDT.write(fileName+'NEDFound.tbl', format='ascii')

Can't identify FITS format in file:  3C234
Can't identify FITS format in file:  Arp299A
Can't identify FITS format in file:  CBS74
Can't identify FITS format in file:  IC0860
Can't identify FITS format in file:  IC4397
Can't identify FITS format in file:  LEDA84081
Can't identify FITS format in file:  LEDA84113
Can't identify FITS format in file:  LEDA84586
Can't identify FITS format in file:  LEDA90159
Can't identify FITS format in file:  LEDA90258
Can't identify FITS format in file:  Mrk34
Can't identify FITS format in file:  Mrk273
Can't identify FITS format in file:  Mrk463
Can't identify FITS format in file:  Mrk477
Can't identify FITS format in file:  Mrk673
Can't identify FITS format in file:  Mrk686
Can't identify FITS format in file:  Mrk1014
Can't identify FITS format in file:  NGC3982
Can't identify FITS format in file:  NGC4579
Can't identify FITS format in file:  NGC4579
Can't identify FITS format in file:  NGC5033
Can't identify FITS format in file:  NGC5273
Can't identif

In [22]:
newResultListDT

ObjID,ra,dec,z,isFits
string264,float32,float32,float64,int8
3C234,150.456,28.7858,0.184925,1
Arp299A,172.14,58.5629,0.010411,1
CBS74,128.106,37.1269,0.092145,1
IC0860,198.765,24.6189,0.011164,1
IC4397,214.495,26.4126,0.014744,1
LEDA84081,209.042,29.0931,0.108451,1
LEDA84113,212.079,29.0798,0.116777,1
LEDA84586,230.659,33.5266,0.124409,1
LEDA90159,180.611,-1.48758,0.150694,1


###     SDSS query from a list file .tbl

In [25]:
fileName = 'list1NEDFound'
newList = Table.read(fileName+".tbl", format = 'ascii')
len(newList)

26

In [26]:
newResultListDT=Table(names=('ObjID','ra','dec','z', 'isFits', 'type'),dtype=('S33', 'f12', 'f12','f8', 'i1', 'S13'))
ListaGalaxias=newList['specobjid']
for i in range(len(ListaGalaxias)):
    name = newList['specobjid'][i]
    # resultNED= Ned.query_object(ListaGalaxias[i])
    RA = newList['ra'][i]  #*u.degree   #resultNED['RA(deg)'][0]*u.degree   
    DEC = newList['dec'][i] #*u.degree   #resultNED['DEC(deg)'][0]*u.degree
    z = newList['redshift'][i]    #resultNED['Redshift'][0]
    pos=coords.SkyCoord(ra=RA, dec=DEC, frame='icrs', unit='deg')  #Cordenadas en sistema ICRS
    try:
        xid = SDSS.query_region(pos, spectro=True)
        if xid is None:
            print i, str(name)+': Not found'
        else:
            sp = SDSS.get_spectra(matches=xid)
            for iii in range(len(sp)):
                hdulist = fits.HDUList(sp[iii])
                total= hdulist[1].data
                linlam=np.exp(total['loglam']*np.log(10))
                lambd = linlam/(1+z)
                if lambd[0]<3710 and lambd[-1]>6760:
                    name = str(ListaGalaxias[i])+'SP'+str(iii+1)
                    print i, str(name)+': Found one spectrum'
                    hdulist.writeto('Exam_fits/'+name+'.fits',clobber=True)
                    if 'bptclass' in newList:
                        if newList['bptclass'][i] == 1:
                            typeGal = 'SF'
                        elif newList['bptclass'][i] == 2:
                            typeGal = 'lowS/N SF'
                        elif newList['bptclass'][i] == 3:
                            typeGal = 'comp'
                        elif newList['bptclass'][i] == 4:
                            typeGal = 'AGN'
                        elif newList['bptclass'][i] == 5:
                            typeGal = 'lowS/N LINER'
                        else:
                            typeGal = 'unclass'
                    else:
                        typeGal = 'unknown'
                    newResultListDT.add_row([name,pos.ra*u.degree,pos.dec*u.degree,z, 1, typeGal])
    except: # HTTPError:
        # e = sys.exc_info()[0]
        # write_to_page( "<p>Error: %s</p>" % e )
        continue

newResultListDT.write(fileName+'SDSSFound.tbl', format='ascii')

0 3C234SP1: Found one spectrum
1 Arp299A: Not found
2 CBS74SP1: Found one spectrum
5 LEDA84081SP1: Found one spectrum
6 LEDA84113SP1: Found one spectrum
7 LEDA84586SP1: Found one spectrum
8 LEDA90159SP1: Found one spectrum
9 LEDA90258: Not found
10 Mrk34SP1: Found one spectrum
11 Mrk273SP1: Found one spectrum
12 Mrk463: Not found
13 Mrk477SP1: Found one spectrum
14 Mrk673SP1: Found one spectrum
16 Mrk1014SP1: Found one spectrum
18 NGC4579: Not found
19 NGC4579: Not found
20 NGC5033: Not found
22 PGC025218SP1: Found one spectrum
23 PGC048898SP1: Found one spectrum
24 PGC055020SP1: Found one spectrum
25 PGC055114SP1: Found one spectrum


In [16]:
newResultListDT

ObjID,ra,dec,z,isFits,type
string264,float32,float32,float64,int8,string104
3C234SP1,150.456,28.7858,0.184925,1,unknown
CBS74SP1,128.106,37.1269,0.092145,1,unknown
Mrk34SP1,158.536,60.0311,0.0505,1,unknown
Mrk273SP1,206.175,55.8868,0.03778,1,unknown
Mrk477SP1,220.159,53.5044,0.037726,1,unknown
PGC055020SP1,231.183,23.6695,0.139,1,unknown
PGC055114SP1,231.748,35.9771,0.055155,1,unknown


In [9]:
newList

col1,col2,col3,col4,col5,col6,col7,col8,col9
string48,float64,float64,float64,string40,string40,string40,float64,float64
8025,53.07059,-27.75539,1.303,0.45,<0.24,1.65,10.4,0.88
6837,53.12799,-27.7714,1.306,>5.82,0.57,0.80,10.45,0.79
9956,53.0909,-27.73119,1.307,>5.61,<0.33,<0.58,9.09,0.57
6408,53.13639,-27.77499,1.308,1.16,<0.22,0.50,9.34,0.58
11132,53.08729,-27.7185,1.308,1.43,0.24,0.19,9.64,0.79
8263,53.02619,-27.7523,1.327,1.47,<0.45,0.60,9.8,0.63
11404d,53.1248,-27.7171,1.356,2.12,0.28,0.33,10.8,0.8
11730,53.1454,-27.7126,1.361,4.10,<0.38,0.73,9.26,0.62
9038e,53.07529,-27.74259,1.374,1.80,0.26,0.24,10.02,0.53


In [2]:
name = 'arco-001'
hdulist = fits.open('crudas/'+name+'.fit')


In [3]:
head= hdulist[0].header
total= hdulist[0].data

In [6]:
hdulist.writeto('crudas/'+name+'SP1.fits',clobber=True)